In [ ]:
!pip install speechbrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.1/760.1 kB 7.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
%%capture
!pip install datasets -U
!pip install librosa
!pip install jiwer

In [ ]:
from speechbrain.inference.speaker import EncoderClassifier
import torch
from collections import defaultdict
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.transforms import Compose
from transformers import WhisperForConditionalGeneration, WhisperProcessor, AutoFeatureExtractor, WhisperModel
from datasets import load_dataset, DatasetDict, Audio

In [ ]:
common_voice = DatasetDict()

common_voice_train = load_dataset("fsicoli/common_voice_17_0", "ja", split="train")
common_voice_test = load_dataset("fsicoli/common_voice_17_0", "ja", split="test")

print(common_voice)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for fsicoli/common_voice_17_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/fsicoli/common_voice_17_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `d

Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 10039it [00:00, 73080.33it/s]


Generating validation split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 6261it [00:00, 56030.61it/s]


...

Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 8732it [00:00, 87307.59it/s]
Reading metadata...: 17463it [00:00, 85923.34it/s]
Reading metadata...: 26547it [00:00, 88144.67it/s]
Reading metadata...: 36379it [00:00, 92137.92it/s]
Reading metadata...: 45597it [00:00, 91172.55it/s]
Reading metadata...: 54719it [00:00, 86165.41it/s]
Reading metadata...: 63383it [00:00, 81264.99it/s]
Reading metadata...: 72168it [00:00, 83212.55it/s]
Reading metadata...: 80551it [00:00, 82412.43it/s]
Reading metadata...: 89097it [00:01, 83310.83it/s]
Reading metadata...: 97460it [00:01, 81727.34it/s]
Reading metadata...: 105658it [00:01, 81548.23it/s]
Reading metadata...: 113830it [00:01, 80686.75it/s]
Reading metadata...: 123522it [00:01, 85442.21it/s]
Reading metadata...: 133276it [00:01, 89017.37it/s]
Reading metadata...: 142484it [00:01, 89924.32it/s]
Reading metadata...: 151493it [00:01, 88197.43it/s]
Reading metadata...: 160522it [00:01, 88811.74it/s]
Reading metadata...: 169417it [00:0

Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 13547it [00:00, 80215.48it/s]


DatasetDict({
    
})


In [ ]:
common_voice_train = common_voice_train.remove_columns(["client_id"])
common_voice_test = common_voice_test.remove_columns(["client_id"])

In [ ]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))

In [ ]:
## ECAPA encoding
classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb")
signal = common_voice_train[0]["audio"]["array"]
# fs = common_voice_train[0]["audio"]["sampling_rate"]
embeddings = classifier.encode_batch(torch.tensor(signal))

## Whisper Encoding
model = WhisperModel.from_pretrained("openai/whisper-base")
feature_extractor = AutoFeatureExtractor.from_pretrained("openai/whisper-base")

inputs = feature_extractor(common_voice_train[0]["audio"]["array"], sampling_rate=common_voice_train[0]["audio"]["sampling_rate"], return_tensors="pt")
input_features = inputs.input_features
decoder_input_ids = torch.tensor([[1, 1]]) * model.config.decoder_start_token_id
last_hidden_state = model(input_features, decoder_input_ids=decoder_input_ids).encoder_last_hidden_state
list(last_hidden_state.shape)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[1, 1500, 512]

In [ ]:
embeddings.shape

torch.Size([1, 1, 192])

Embedding shape from Whisper: Batch X 1500 X 512

Embedding shape from ECAPA: Batch X 1 X 192

In [ ]:
# Define your downsampling convolutional layer
downsample_conv = nn.Conv1d(in_channels=512, out_channels=64, kernel_size=1)

# Assuming your input tensor is named input_tensor
# Reshape the tensor to fit the convolutional layer
input_tensor = torch.ones(2,1500,512)
input_tensor = input_tensor.permute(0, 2, 1)  # Change shape to batch * 512 * 1500

# Apply convolution
output_tensor = downsample_conv(input_tensor)

# Check the shape
print(output_tensor.shape)  # Should be batch * 64 * 500


torch.Size([2, 64, 1500])


In [ ]:
class SpeechModel(torch.nn.Module):
    def __init__(self, whisper_model, ecapa_model):
        super(SpeechModel, self).__init__()
        self.sampling_rate = 16000
        self.whisper_encoder = WhisperModel.from_pretrained("openai/whisper-base")
        self.whisper_feature_extractor = AutoFeatureExtractor.from_pretrained("openai/whisper-base")
        self.ecapa_encoder = EncoderClassifier.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb")

        # Define downsampling layers
        self.whisper_downsample = nn.Conv1d(in_channels=512, out_channels=64, kernel_size=1)
        self.ecapa_downsample = nn.Conv1d(in_channels=192, out_channels=64, kernel_size=1)

        # Define Transformer layers
        self.transformer_encoder = nn.TransformerEncoderLayer(d_model=128, nhead=8, dim_feedforward=512, batch_first=True)
        self.transformer_decoder = nn.TransformerDecoderLayer(d_model=128, nhead=8, dim_feedforward=512, batch_first=True)

        # Define prediction heads
        #self.next_word_prediction_head = nn.Linear(...)
        #self.speaker_recognition_head = nn.Linear(...)

    def forward(self, x):
        # Forward pass for whisper branch
        x_whisper = self.whisper_feature_extractor(x, sampling_rate=self.sampling_rate, return_tensors="pt")
        whisper_embedding = self.whisper_encoder(x_whisper).input_features
        downsampled_whisper = self.whisper_downsample(whisper_embedding)

        # Forward pass for ECAPA branch
        ecapa_embedding = self.ecapa_encoder(x)
        downsampled_ecapa = self.ecapa_downsample(ecapa_embedding)

        # Concatenate downscaled embeddings
        concatenated_embeddings = torch.cat((downsampled_whisper, downsampled_ecapa), dim=1)

        # Transformer layers
        transformer_output = self.transformer_encoder(concatenated_embeddings)

        # Task-specific heads
        next_word_prediction = self.next_word_prediction_head(transformer_output)
        speaker_recognition = self.speaker_recognition_head(transformer_output)

        return next_word_prediction, speaker_recognition

# Define model
whisper_model = OpenAIWhisperEncoder(...)
ecapa_model = ECAPATDNNModel(...)
model = SpeechModel(whisper_model, ecapa_model)

# Define loss functions
next_word_loss_function = nn.CrossEntropyLoss()
speaker_recognition_loss_function = nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define data preprocessing
transform = Compose([preprocess_function])

# Load dataset
dataset = CommonVoiceDataset(root_dir='path_to_commonvoice_dataset', transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for batch in dataloader:
        optimizer.zero_grad()
        inputs, targets = batch
        next_word_prediction, speaker_recognition = model(inputs)

        # Calculate loss
        next_word_loss = next_word_loss_function(next_word_prediction, targets)
        # Calculate speaker recognition loss
        speaker_recognition_loss = speaker_recognition_loss_function(speaker_recognition, speaker_labels)

        total_loss = next_word_loss + speaker_recognition_loss

        # Backpropagation
        total_loss.backward()
        optimizer.step()
